In [ ]:
%pip install semantic-link-labs==0.11.1

In [ ]:
%run Utils

In [ ]:
config_dict = get_config_dict()

#create the sub-directory in Files folder where the CSV files will be downloaded
lakehouse_dir = "Files/cmsdemofiles/CMS Medicare Part D Star Schema.Report"
lakehouse_dir_abfss_full_path = get_full_abfss_path(config_dict['workspace_id'], config_dict['lakehouse_id'], lakehouse_dir)

#notebookutils.fs.mkdirs(lakehouse_dir_abfss_full_path)

print(f"lakehouse_dir: {lakehouse_dir}")
print(f"lakehouse_dir_abfss_full_path: {lakehouse_dir_abfss_full_path}")

In [ ]:
#local mount path is required for plain python to save files to Lakehouse file section
mount_point = "/mnt/lakehouse/" + config_dict["lakehouse_name"] + "/" + lakehouse_dir
lakehouse_dir_local_mount_path = mount_path_return_local_path(lakehouse_dir_abfss_full_path, mount_point)

print(f'mount point: {mount_point}')
print(f"lakehouse_dir_local_path: {lakehouse_dir_local_mount_path}")

In [ ]:
import os

report_parts = {}

for root, dirs, files in os.walk(lakehouse_dir_local_mount_path):
    print(f'Current directory: {root}')
    for file in files:
        print(f'  File: {file}')
        part_key = root.replace(lakehouse_dir_local_mount_path, "") + "/" + file
        part_key = part_key.lstrip('/')
        print(f'part_key: {part_key}')

        with open( root + "/" + file, "r", encoding="utf-8") as file:
            payload = file.read()
        
        report_parts[part_key] = payload
        
 
    for dir_name in dirs:
        print(f'  Subdirectory: {dir_name}')


In [ ]:
import base64

request_body = {
    "displayName": "cms_report",
    "description": "cms report imported using API",
    "definition" : {
        "parts": []
    }
}

In [ ]:


for key, value in report_parts.items():

    new_part = {
        "path": key,
        "payload" : base64.b64encode(value.encode('utf-8')),
        "payloadType": "inlineBase64"
    }
    
    request_body["definition"]["parts"].append(new_part)

In [ ]:
# Welcome to your new notebook
# Type here in the cell editor to add code!
import sempy.fabric as fabric
from sempy.fabric.exceptions import FabricHTTPException
import json
import fnmatch
import re
import os


# https://learn.microsoft.com/en-us/python/api/semantic-link-sempy/sempy.fabric?view=semantic-link-python
# FabricRestClient uses the default credentials of the executing user
client = fabric.FabricRestClient()



In [ ]:
workspace_id = notebookutils.runtime.context["currentWorkspaceId"]
print(workspace_id)

create_report_uri = f"v1/workspaces/{workspace_id}/reports"
print(create_report_uri)

In [ ]:
#workspace_id = config_dict['workspace_id']
response = client.request(method = "POST", path_or_url=create_report_uri, lro_wait=True, json = request_body)
print(response.status_code)
print(response.text)

In [ ]:
import sempy_labs.report as rep

report = "cms_report"
semantic_model = "cms_semantic_model2"
#report_workspace = "CMSDemo-0125"
#sematic_model_workspace = "CMSDemo-0125"

rep.report_rebind(report=report, dataset=semantic_model)